In [42]:
import numpy as np
from matplotlib import pyplot as plt
import os
import h5py
import pickle
import re
import pdb
from collections import defaultdict, Counter
%matplotlib inline

In [43]:
path = '/dfs/scratch0/wleif/Reddit/clean_comments/'
subreddits = os.listdir(path)
subreddits = ['funny', 'leagueoflegends', 'AdviceAnimals', 'pics', 'nfl']

In [ ]:
#%%timeit -n1 -r1
print_every = 1

usr2ind = {}
ind2usr = {}
n_users = 0

sub2ind = {}
ind2sub = {}
n_subs = len(subreddits)

users = {}

'''
sample = np.random.random_integers(0, len(subreddits), nsubs)
print sampled_subreddits
for i, sub_idx in enumerate(sample):
    subreddit = subreddits[sub_idx]
    sub2ind[subreddit] = sub_idx
    ind2sub[sub_idx] = subreddit
'''

for i, subreddit in enumerate(subreddits):
    sub2ind[subreddit] = i
    ind2sub[i] = subreddit
    with open(path+subreddit+'.tsv', 'r') as df:
        #raw_data = np.genfromtxt(df, dtype=str, delimiter='\t')
        for row in df:
            user = row.split('\t')[-5]
            if user not in usr2ind:
                ind2usr[n_users] = user
                usr2ind[user] = n_users
                users[n_users] = defaultdict(int)
                n_users += 1
            users[usr2ind[user]][i] += 1

    
    if i % print_every == 0:
        print "Finished %d" % (i+1)

Finished 1


In [30]:
'''

Some analytics

'''

counter = [0.0] * n_subs
r_counter = [0.0] * n_subs
n_posts = 0
thresh = 3
user_pool = []

def robust_count(d, n_spans = 5):
    return sum([1 if v > n_spans else 0 for v in [v for v in d.values()] ])
        
for k, v in users.iteritems():
    n_posts += sum([uv for _, uv in v.iteritems()])
    for i in xrange(n_subs):
        if len(v) > i:
            counter[i] += 1
        if robust_count(v) > i:
            r_counter[i] += 1
    if robust_count(v) > thresh:
        user_pool.append(k)
percents = [n / n_users for n in r_counter]
for i, percent in enumerate(percents):
    print "nsubs: %d, percent: %.3f, robust count: %d, full count: %d" % (i, percent, r_counter[i], counter[i])

user_pool = set(user_pool)
print "Total number of users: %d, Number of selected users: %d" % (n_users, len(user_pool))
print "Total number of posts: %d" % n_posts


nsubs: 0, percent: 0.302, robust count: 586041, full count: 1939091
nsubs: 1, percent: 0.125, robust count: 242105, full count: 898131
nsubs: 2, percent: 0.059, robust count: 114459, full count: 433442
nsubs: 3, percent: 0.007, robust count: 13687, full count: 67131
nsubs: 4, percent: 0.000, robust count: 505, full count: 3684
Total number of users: 1939091, Number of selected users: 13687
Total number of posts: 40737131


In [31]:
'''

Get the posts for the users that selected population of users (user_pool)

'''

print_every = [1.,1.]
posts = [[[] for x in xrange(n_users)] for y in xrange(n_subs)]

try:
    for i, subreddit in enumerate(subreddits):
        nrows = sum(1 for row in open(path+subreddit+'.tsv', 'r'))
        with open(path+subreddit+'.tsv', 'r') as df:
            sub_ind = sub2ind[subreddit]
            for j, row in enumerate(df):
                data = row.split('\t')
                user = usr2ind[data[-5]]
                if user in user_pool:
                    posts[sub_ind][user].append((data[-1], data[1]))

                if j % int(nrows/print_every[0]) == 0: ## TODO: Fix logging...
                    print '\t%.2f' % (j/float(nrows))

        if i % print_every[1] == 0:
            print "Finished %d" % (i+1)
except Exception as e:
    pdb.set_trace()

	0.00
	0.20
	0.40
	0.60
	0.80
	1.00
Finished 1
	0.00
	0.20
	0.40
	0.60
	0.80
Finished 2
	0.00
	0.20
	0.40
	0.60
	0.80
	1.00
Finished 3
	0.00
	0.20
	0.40
	0.60
	0.80
	1.00
Finished 4
	0.00
	0.20
	0.40
	0.60
	0.80
	1.00
Finished 5


In [32]:
print [0 for x in xrange(3)]

[0, 0, 0]


In [41]:
'''

Build, prune, and write vocab

'''

min_doc_appearances = 4
remove_top_k = 500
max_vocab_size = 10000
outfile = '/dfs/scratch0/wangalex/rmn/reddit5'

freqs = defaultdict(int)
doc_freqs = {}
special = ['<EOS>', '<UNK>', '<SPECIAL>', '<URL>']
lens = []
max_len = 0

print "Sorting by date and gathering vocab..."
for i in xrange(n_subs):
    for j in xrange(n_users):
        if posts[i][j] is []:
            continue
        posts[i][j].sort(key=lambda tup:tup[-1])
        for post,_ in posts[i][j]:
            clean_words = re.sub(r"[0-9]{1,}", "<NUM>", re.sub(r"<(.+?)>", "", post)).strip().split()
            for word in clean_words:
                if word not in doc_freqs:
                    doc_freqs[word] = [0 for x in xrange(nsubs)]
                doc_freqs[word][i] = 1
                freqs[word] += 1
            lens.append(len(clean_words))
            max_len = max(max_len, len(clean_words))

# pruning vocab borrowed from Yoon Kim
print 'Pruning vocab...'
vocab = [(word, count) for word, count in freqs.iteritems()]
vocab.sort(key = lambda x: x[1], reverse = True)
doc_freqs = {word:sum(freq) for word, freq in doc_freqs.iteritems()}
no_common_words = [pair[0] for pair in vocab[remove_top_k:]]
min_doc_words = filter(lambda x: doc_freqs[x] >= min_doc_appearances, no_common_words)
pruned = min_doc_words[:max_vocab_size-len(special)]

word2ind = {}
ind2word = {}
ind = 0
for word in special+pruned:
    word2ind[word] = ind
    ind2word[ind] = word
    ind += 1

print "Writing vocab..."
with open(outfile+'.vocab.txt', 'w') as f:
    f.write("Word Index Count DocFreq\n")
    words = [(word, idx) for word, idx in word2ind.iteritems()]
    words.sort(key = lambda x: x[1])
    for word, idx in words:
        if word in freqs:
            f.write("%s %d %d %d\n" % (word, idx, freqs[word], doc_freqs[word]))
        else:
            f.write("%s %d %d %d\n" % (word, idx, -1, -1))

with open(outfile+'.vocab.pkl', 'w') as f:
    pickle.dump((word2ind, ind2word), f)

print '\tFull vocab size: %d, pruned vocab size: %d' % (len(vocab), len(word2ind))

Sorting by date and gathering vocab...
Pruning vocab...
Writing vocab...
	Full vocab size: 519523, pruned vocab size: 10000


In [ ]:
'''

Build word2vec pretrained embeddings

'''

word_vecs = {}
vec_file = '/dfs/scratch0/wangalex/GoogleNews-vectors-negative300.bin.gz'
with open(vec_file, "rb") as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in xrange(vocab_size):
        word = []
        while True:
            ch = f.read(1)
            if ch == ' ':
                word = ''.join(word)
                break
            if ch != '\n':
                word.append(ch)
        if word in vocab:  
            word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')
        else:
            f.read(binary_len)

embed = np.random.uniform(-0.25, 0.25, (len(vocab), len(word_vecs.values()[0])))
embed[0] = 0
for word, vec in word_vecs.items():
    embed[vocab[word]] = vec
print "\tLoaded %d vectors" % len(word_vecs)


In [ ]:
posts = []

for i in xrange(n_subs):
    sub = np.array([i], dtype=np.int32)
    for j in xrange(n_users):
        if posts[i][j] is []:
            continue
        user = np.array([j], dtype=np.int32)
        spans = []
        masks = []
        for post,_ in posts[i][j]:
            clean_words = re.sub(r"[0-9]{1,}", "<NUM>", re.sub(r"<(.+?)>", "", post)).strip().split()
            span = [word2ind[word] if word in word2ind else word2ind['UNK'] for word in clean_words[:max_len]] + /
                    [-1]*(max_len-len(clean_words))
            mask = (span >= 0).astype(int)
            spans.append(span)
            masks.append(mask)
    posts.append([sub, user, np.array(spans, dtype=np.int32), np.array(mask, dtype=np.int32)])